# Prvi domaći zadatak iz predmeta Digitalna obrada slike (OE4DOS) 2021/2022
Aleksa Cvetanović 2018/0160


# Zadatak 1

U prvom zadatku potrebno je realizovati funkciju *dress_queen_still(pattern)* koja formira binarnu masku oblika haljine engleske kraljice, i pomoću nje menja njen dezen. Takođe, potrebno je prikazati sve međurezultate, i na osnovu njih argumentovati izbor metoda korišćenih za obradu ove slike. Na kraju, potrebno je izvršiti dodatnu obradu slike sa promenjenim dezenom, radi što realnijeg izgleda.

## Uvoz modula
Za obradu slike korišćena je biblioteka *skimage*, kao i prateće biblioteke *numpy* za matematička izračunavanja, i *matplotlib* za iscrtavanje grafika i slika.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color, filters, util

## Formiranje maske
Formiranje maske sastoji se iz nekoliko koraka:
1. Potrebno je izabrati kolor sistem iz koga se najlakše mogu izvući potrebne informacije
2. Potrebno je izvršiti obradu određenih ravni slike u izabranom kolor sistemu kako bi se uspešnije detektovali željeni pikseli
3. Potrebno je odrediti opsege vrednosti u kojima se nalaze intenziteti odgovarajućih ravni željenih piksela

Za jednostavniji prikaz ulazne slike u različitim kolor sistemima korišćena je funkcija *showPlanes(img, color_system)*, koja za ulaznu sliku i kolor sistem daje prikaz svih ravni te slike pojedinačno.

In [ ]:
def showPlanes(img, color_system):
    fig, axs = plt.subplots(1, 3, figsize=(15,8), dpi=80)
    plt.tight_layout()
    plt.suptitle(color_system + ' Color System', fontsize=20)
    axs[0].imshow(img[:,:,0], cmap='jet')
    axs[0].set_title(color_system[0] + ' plane')
    axs[0].set_axis_off()
    axs[1].imshow(img[:,:,1], cmap='jet')
    axs[1].set_title(color_system[1] + ' plane')
    axs[1].set_axis_off()
    axs[2].imshow(img[:,:,2], cmap='jet')
    axs[2].set_title(color_system[2] + ' plane')
    axs[2].set_axis_off()

Sledeći deo koda radi učitavanje originalne slike, konvertuje je iz RGB u različite kolor sisteme, i prikazuje ih po ravnima.

In [ ]:
# Read image
data_dir = '../sekvence'
imgRGB = io.imread(data_dir + '/queen_dress.jpg')

# Convert to different color systems
imgHSV = color.rgb2hsv(imgRGB)
imgYUV = color.rgb2yuv(imgRGB)
imgLab = color.rgb2lab(imgRGB)

# Plot image planes
showPlanes(imgRGB, 'RGB')
showPlanes(imgHSV, 'HSV')
showPlanes(imgYUV, 'YUV')
showPlanes(imgLab, 'Lab')

Sa priloženih slika očigledno je da se maska najbolje može izvući iz H ravni slike u HSV kolor sistemu. Međutim, problem sa ovako definisanom maskom jeste taj što će ona obuhvatiti i neke piksele uz desnu ivicu slike (cveće slične boje kao kraljičina haljina). Pored nje, može se iskoristiti i S ravan slike, kojom se detektuje obojenost. Pojedinačna primena ovih maski ne dovodi do željenih rezultata, pa je najbolje formirati novu masku na osnovu obe ravni, primenom logičke operacije "I" na gore navedene maske. Rezultat ove operacije smešten je u promenljivu *maskStart*. Dobijena maska još uvek nije dovoljno dobra, ali predstavlja dobar početak za dalju obradu.

U sledećem segmentu koda prikazane su različite maske, gde su opsezi intenziteta piksela određeni na osnovu vrednosti piksela sa slika odozgo. Najbolji opsezi intenziteta će biti određeni kasnije.

In [ ]:
# Define starting mask
maskH = (imgHSV[:,:,0]>0.3) & (imgHSV[:,:,0]<0.6)
maskS = (imgHSV[:,:,1]>0.7) & (imgHSV[:,:,1]<0.85)
maskStart = maskH & maskS

# Plot results
fig, axs = plt.subplots(1, 3, figsize=(15,8), dpi=100)
plt.tight_layout()
axs[0].imshow(maskH.astype(np.float64), vmin=0, vmax=1, cmap='gray')
axs[0].set_title('H plane mask')
axs[0].set_axis_off()
axs[1].imshow(maskS.astype(np.float64), vmin=0, vmax=1, cmap='gray')
axs[1].set_title('S plane mask')
axs[1].set_axis_off()
axs[2].imshow(maskStart.astype(np.float64), vmin=0, vmax=1, cmap='gray')
axs[2].set_title('Both masks');
axs[2].set_axis_off()

## Poboljšavanje binarne maske
Problem pojave neželjenih piksela uz desnu ivicu slike je rešen primenom 2 različite maske, definisane na osnovu H i S ravni slike u HSV kolor sistemu. Međutim, sa prikaza iznad se vidi da maska dobijena iz S ravni kvari kvalitet maske u oblasti kraljičine haljine - neke piksele koji pripadaju haljini uopšte ne registruje, zbog njihove velike razlike u intenzitetu. Ovaj problem može se rešiti primenom Gausovog filtra na S ravan slike, čime bi se intenzitet tih piksela približio vrednostima okolnih piksela.

U sledećem delu koda testirane su 3 različita Gausova filtra, i prikazani su njihovi rezultati.

In [ ]:
# Apply Gaussian blur to S plane
imgHSVb5 = imgHSV.copy()
imgHSVb15 = imgHSV.copy()
imgHSVb27 = imgHSV.copy()
imgHSVb41 = imgHSV.copy()
imgHSVb5[:,:,1] = filters.gaussian(imgHSV[:,:,1], sigma=1, truncate=5) # 5x5 Gaussian filter
imgHSVb15[:,:,1] = filters.gaussian(imgHSV[:,:,1], sigma=4, truncate=3.75) # 15x15 Gaussian filter
imgHSVb27[:,:,1] = filters.gaussian(imgHSV[:,:,1], sigma=15, truncate=1.8) # 27x27 Gaussian filter
imgHSVb41[:,:,1] = filters.gaussian(imgHSV[:,:,1], sigma=10, truncate=4.1) # 41x41 Gaussian filter

# Plot results
fig, axs = plt.subplots(1, 4, figsize=(15,8), dpi=100)
plt.tight_layout()
axs[0].imshow(imgHSVb5[:,:,1], vmin=0, vmax=1, cmap='gray')
axs[0].set_title('5x5 Gaussian filter')
axs[0].set_axis_off()
axs[1].imshow(imgHSVb15[:,:,1], vmin=0, vmax=1, cmap='gray')
axs[1].set_title('15x15 Gaussian filter')
axs[1].set_axis_off()
axs[2].imshow(imgHSVb27[:,:,1], vmin=0, vmax=1, cmap='gray')
axs[2].set_title('27x27 Gaussian filter');
axs[2].set_axis_off()
axs[3].imshow(imgHSVb41[:,:,1], vmin=0, vmax=1, cmap='gray')
axs[3].set_title('41x41 Gaussian filter');
axs[3].set_axis_off()

# Create masks after applying blur
maskSb5 = (imgHSVb5[:,:,1]>0.7) & (imgHSVb5[:,:,1]<0.85)
maskSb15 = (imgHSVb15[:,:,1]>0.7) & (imgHSVb15[:,:,1]<0.85)
maskSb27 = (imgHSVb27[:,:,1]>0.7) & (imgHSVb27[:,:,1]<0.85)
maskSb41 = (imgHSVb41[:,:,1]>0.7) & (imgHSVb41[:,:,1]<0.85)

maskHSb5 = maskH & maskSb5
maskHSb15 = maskH & maskSb15
maskHSb27 = maskH & maskSb27
maskHSb41 = maskH & maskSb41

# Plot results
fig, axs = plt.subplots(1, 4, figsize=(15,8), dpi=100)
plt.tight_layout()
axs[0].imshow(maskHSb5.astype(np.float64), vmin=0, vmax=1, cmap='gray')
axs[0].set_title('Mask with 5x5 Gaussian filter')
axs[0].set_axis_off()
axs[1].imshow(maskHSb15.astype(np.float64), vmin=0, vmax=1, cmap='gray')
axs[1].set_title('Mask with 15x15 Gaussian filter')
axs[1].set_axis_off()
axs[2].imshow(maskHSb27.astype(np.float64), vmin=0, vmax=1, cmap='gray')
axs[2].set_title('Mask with 27x27 Gaussian filter');
axs[2].set_axis_off()
axs[3].imshow(maskHSb41.astype(np.float64), vmin=0, vmax=1, cmap='gray')
axs[3].set_title('Mask with 41x41 Gaussian filter');\
axs[3].set_axis_off()

Sa slika odozgo vidi se da Gausov filtar dimenzija 27x27 daje najbolje rezultate pri kreiranju maske. Filtri manjih dimenzija ne uspevaju da dovoljno ujednače problematične intenzitete piksela da bi oni bili registrovani od strane maske. Za dalje razmatranje biće koriščena maska *maskHSb27*, koja predstavlja rezultat operacije logičkog "I" nad maskom *maskH*, dobijenom iz H ravni slike, i maskom *maskSb27*, dobijenom iz S ravni slike na koju je primenjen Gausov filtar dimenzija 27x27.

## Određivanje opsega intenziteta piksela u H i S ravni
Korišćeni opsezi uspevali su da daju solidne rezultate, ali su izabrani bez konkretne ideje tačnom opsegu vrednosti koje kraljičina haljina u ovim ravnima ima. Ovi opsezi mogu se uočiti iz histograma slike, pri čemu se traži pik u odgovarajućem delu histograma.

Sledeći deo koda određuje i iscrtava histograme H i S ravni slike.

In [ ]:
histImgInH, binEdges = np.histogram(255*imgHSV[:,:,0].flatten(), bins=256, range=(0,255)) # H plane histogram
histImgInS, binEdges = np.histogram(255*imgHSVb27[:,:,1].flatten(), bins=256, range=(0,255)) # S plane histogram

# Plot histograms
fig, axs = plt.subplots(1,2, figsize=(20,6), dpi=100)
axs[0].bar(binEdges[0:-1], histImgInH)
axs[0].set_title('H plane histogram')
axs[1].bar(binEdges[0:-1], histImgInS)
axs[1].set_title('S plane histogram');

Histogram H ravni jasno pokazuje pik na vrednosti koja označava intenzitet kraljičine haljine. Ovaj veoma izraženi pik odgovara ranije prikazanoj slici H ravni i gotovo jednakim intenzitetima piksela haljine. Sa histograma se za opseg intenziteta piksela u H ravni mogu očitati vrednosti opsega [100, 125], odnosno opsega [0.39, 0.49] nakon skaliranja na opseg [0, 1].

Zbog veoma dobre maske *maskH* može se uzeti širi opseg intenziteta piksela S ravni. Na taj način se mogu obuhvatiti gotovo svi pikseli haljine, bez da se obuhvate problematični pikseli na desnoj strani slike.

U sledećem delu koda postavljeni su novi opsezi intenziteta piksela za obe maske, i izvršeno je poređenje rezultata za različite opsege intenziteta piksela S ravni.

In [ ]:
# Create masks with various pixel intensity range
maskH = (imgHSV[:,:,0]>0.39) & (imgHSV[:,:,0]<0.49)
maskS1 = (imgHSVb27[:,:,1]>=0.2) & (imgHSVb27[:,:,1]<=0.85)
maskS2 = (imgHSVb27[:,:,1]>=0.4) & (imgHSVb27[:,:,1]<=0.85)
maskS3 = (imgHSVb27[:,:,1]>=0.476) & (imgHSVb27[:,:,1]<=0.85)
maskS4 = (imgHSVb27[:,:,1]>=0.55) & (imgHSVb27[:,:,1]<=0.85)

mask1 = maskH & maskS1
mask2 = maskH & maskS2
mask3 = maskH & maskS3
mask4 = maskH & maskS4

# Plot masks
fig, axs = plt.subplots(1,4,figsize=(20,8), dpi=80)
plt.tight_layout()
axs[0].imshow(mask1.astype(np.float64), cmap='gray')
axs[0].set_title('S plane range: [0.2, 0.85]', fontsize=20)
axs[0].set_axis_off()
axs[1].imshow(mask2.astype(np.float64), cmap='gray')
axs[1].set_title('S plane range: [0.4, 0.85]', fontsize=20)
axs[1].set_axis_off()
axs[2].imshow(mask3.astype(np.float64), cmap='gray')
axs[2].set_title('S plane range: [0.476, 0.85]', fontsize=20)
axs[2].set_axis_off()
axs[3].imshow(mask4.astype(np.float64), cmap='gray');
axs[3].set_title('S plane range: [0.55, 0.85]', fontsize=20)
axs[3].set_axis_off()

# Apply masks
imgMask1 = imgRGB.copy()
imgMask2 = imgRGB.copy()
imgMask3 = imgRGB.copy()
imgMask4 = imgRGB.copy()

imgMask1[mask1] = [255, 255, 255]
imgMask2[mask2] = [255, 255, 255]
imgMask3[mask3] = [255, 255, 255]
imgMask4[mask4] = [255, 255, 255]

# Plot images with applied masks
fig, axs = plt.subplots(1,4,figsize=(20,8), dpi=80)
plt.tight_layout()
axs[0].imshow(imgMask1, cmap='gray')
axs[0].set_title('S plane range: [0.2, 0.85]', fontsize=20)
axs[0].set_axis_off()
axs[1].imshow(imgMask2, cmap='gray')
axs[1].set_title('S plane range: [0.4, 0.85]', fontsize=20)
axs[1].set_axis_off()
axs[2].imshow(imgMask3, cmap='gray');
axs[2].set_title('S plane range: [0.476, 0.85]', fontsize=20)
axs[2].set_axis_off()
axs[3].imshow(imgMask4, cmap='gray');
axs[3].set_title('S plane range: [0.55, 0.85]', fontsize=20)
axs[3].set_axis_off()

mask = mask2

Najbolja od gore tri prikazane binarne maske jeste *mask2* , koja gotovo u potpunosti uklanja pozadinu sa slike, a istovremeno prekriva najveći deo kraljičine haljine.

## Promena dezena haljine
Nakon formiranja odgovarajuće maske potrebno je istu iskoristiti za promenu dezena kraljičine haljine. Jednostavnom primenom maske može se izvršiti blendovanje originalne slike i novog dezena. Da bi se ista maska mogla primeniti na obe slike, potrebno je da one budu istih dimenzija.

Sledeći deo koda učitava novi dezen, vrši blendovanje dve slike i prikazuje rezultat.

In [ ]:
# Read pattern
dress = io.imread('blue_fabric.jpg')

# Apply mask
imgNewDress = imgRGB.copy()
imgNewDress[mask] = dress[mask]
plt.figure(figsize=(10, 8), dpi=80)
plt.title('Simple blending')
io.imshow(imgNewDress)
plt.axis('off');

Ovaj rezultat nije zadovoljavajući jer se u potpunosti prekrivaju nabori haljine, što čini da nova haljina izgleda potpuno veštački. Da bi se ovaj problem rešio, neophodno je preneti detalje haljine sa originalne slike na novi dezen. Detalji se sa originalne na novu sliku mogu preneti preko Y ravni YUV kolor sistema, koja predstavlja monohromatsku sliku sa velikim brojem detalja. Pored detalja haljine, moraju ostati i detalji koji se nalaze na novom dezenu, pa je potrebno izvršiti blendovanje ove dve slike, uz skaliranje vrednosti intenziteta piksela njihovih Y ravni.

Treba naglasiti da faktori skaliranja u zbiru treba da imaju vrednost 1, kako ne bi došlo do prekoračenja intenziteta piksela Y ravni van opsega [0, 1]. Iako su faktori izabrani tako da ispunjavaju ovaj uslov, zbog zaokruživanja brojeva može se desiti da neke vrednosti prelaze gornju granicu, pa je zbog toga za svaki slučaj primenjena funkcija *np.clip* koja sve intenzitete skalira na odgovarajući opseg vrednosti.

U sledećem delu koda izvršeno je blendovanje opisano u tekstu iznad, i prikazani su rezultati za različite faktore skaliranja detalja originalne slike i detalja novog dezena. Dalje, primenom ranije formirane maske, rezultati blendovanja preneti su na kraljičinu haljinu, i prikazani su finalni rezultati.

In [ ]:
dressYUV=color.rgb2yuv(dress) # Convert RGB to YUV color system


dressYUV1 = dressYUV.copy()
dressYUV2 = dressYUV.copy()
dressYUV3 = dressYUV.copy()

# Apply mask to dress using different scala factors
dressYUV1[:,:,0] = imgYUV[:,:,0]*0.5 + dressYUV[:,:,0]*0.5
dressYUV2[:,:,0] = imgYUV[:,:,0]*0.7 + dressYUV[:,:,0]*0.3
dressYUV3[:,:,0] = imgYUV[:,:,0]*0.8 + dressYUV[:,:,0]*0.2

dressDetails1 = color.yuv2rgb(dressYUV1)
dressDetails2 = color.yuv2rgb(dressYUV2)
dressDetails3 = color.yuv2rgb(dressYUV3)


dressDetails1 = util.img_as_ubyte(np.clip(dressDetails1, 0, 1))
dressDetails2 = util.img_as_ubyte(np.clip(dressDetails2, 0, 1))
dressDetails3 = util.img_as_ubyte(np.clip(dressDetails3, 0, 1))


# Plot dress patterns with original dress details
fig, axs = plt.subplots(1, 3, figsize=(20,8), dpi=100)
plt.tight_layout()
axs[0].imshow(dressDetails1)
axs[0].set_title('Factor ratio: 1', fontsize=20)
axs[0].set_axis_off()
axs[1].imshow(dressDetails2)
axs[1].set_title('Factor ratio: 2.33', fontsize=20)
axs[1].set_axis_off()
axs[2].imshow(dressDetails3)
axs[2].set_title('Factor ratio: 4', fontsize=20)
axs[2].set_axis_off()

# Blend new dress patterns with original image
imgDetails1 = imgRGB.copy()
imgDetails2 = imgRGB.copy()
imgDetails3 = imgRGB.copy()

imgDetails1[mask] = dressDetails1[mask]
imgDetails2[mask] = dressDetails2[mask]
imgDetails3[mask] = dressDetails3[mask]

fig, axs = plt.subplots(1, 3, figsize=(20,8), dpi=100)
plt.tight_layout()
axs[0].imshow(imgDetails1)
axs[0].set_title('Factor ratio: 1', fontsize=20)
axs[0].set_axis_off()
axs[1].imshow(imgDetails2)
axs[1].set_title('Factor ratio: 2.33', fontsize=20)
axs[1].set_axis_off()
axs[2].imshow(imgDetails3)
axs[2].set_title('Factor ratio: 4', fontsize=20);
axs[2].set_axis_off()

Na osnovu prikazanih rezultata vidi se da nema mnogo velikih razlika u zavisnosti od izabranih faktora skaliranja. Iako su razlike male, čini se da druga slika izgleda najrealnije, pa su za faktore skaliranja izabrane vrednosti 0.7 i 0.3. Treba napomenuti da će u zavisnosti od izbora dezena varirati i adekvatni opseg vrednosti, na sledeći način: ukoliko na novom dezenu ima više detalja, treba smanjiti faktor skaliranja originalne slike i povećati faktor skaliranja dezena. Sa druge strane, ukoliko ima manje detalja na dezenu, rezultati će biti zadovoljavajući ukoliko se koristi i manji faktor skaliranja dezena.

## Implementacija funkcije *dress_queen_still(pattern)*

Rezultati prikazani u dosadašnjem izlaganju implementovani su unutar funkcije *dress_queen_still(pattern)*, tražene u zadatku. Kao argument funkcije uzima se dezen u koji želimo da promenimo kraljičinu haljinu, čiji je jedini zahtev da bude istih dimenzija kao i originalna slika, kako bi se formirana maska mogla adekvatno primeniti na obe slike. Ova funkcija definisana je u sledećem delu koda.

In [ ]:
def dress_queen_still(pattern):
    """
        Change Queen's coat to desired pattern.
        
        Parameters
        ----------
        pattern: string
            Path to input pattern. Pattern dimensions are required to be (1080, 816), same as the original image.
        
        Returns
        -------
        newQueenDress: ndarray
            Image of the Queen wearing new coat.
        
    """
    
    # Import original image and convert to color systems required for future use
    data_dir = '../sekvence'
    imgRGB = io.imread(data_dir + '/queen_dress.jpg')
    imgHSV = color.rgb2hsv(imgRGB)
    imgYUV = color.rgb2yuv(imgRGB)
    
    # Apply Gaussian filter to S plane of the original image
    imgHSV[:,:,1] = filters.gaussian(imgHSV[:,:,1], sigma=15, truncate=1.8) # 27x27 Gaussian filter
    
    # Define binary mask
    maskH = (imgHSV[:,:,0]>0.39) & (imgHSV[:,:,0]<0.49)
    maskS = (imgHSV[:,:,1]>0.4) & (imgHSV[:,:,1]<0.85)
    mask = maskH & maskS
    
    # Import pattern and blend it with original image
    dressRGB = io.imread(pattern)
    dressYUV = color.rgb2yuv(dressRGB)
    dressYUV[:,:,0] = imgYUV[:,:,0]*0.7 + dressYUV[:,:,0]*0.3
    dressDetails = color.yuv2rgb(dressYUV)
    dressDetails = util.img_as_ubyte(np.clip(dressDetails, 0, 1))
    
    # Apply binary mask
    newQueenDress = imgRGB.copy()
    newQueenDress[mask] = dressDetails[mask]
    
    return newQueenDress

# Use function dress_queen_still
out1 = dress_queen_still('blue_fabric.jpg')
fig, axs = plt.subplots(1,2,figsize=(15, 8), dpi=80)
plt.tight_layout()
axs[0].imshow(imgRGB)
axs[0].set_title('Original image', fontsize=20)
axs[0].set_axis_off()
axs[1].imshow(out1)
axs[1].set_title('Processed image', fontsize=20)
axs[1].set_axis_off()


# Save resulting image
io.imsave('new_queen_dress.jpg', out1)


# Obrada video snimka
U drugom delu prvog zadatka potrebno je primeniti ideje iz prvog dela na obradu video snimka pomoću funkcije *dress_queen_video*. U ovom slučaju potrebno je obratiti posebnu pažnju na vreme izvršavanja funkcije, zbog velikog broja frejmova video snimka.

Za obradu video snimka primenjen je veoma sličan postupak u odnosu na onaj primenjen na sliku. Potrebno je bilo uvesti dve izmene:
1. Potrebno je na osnovu statističkih parametara slike izabrati samo odgovarajuće frejmove koji će biti obrađeni, da bi se izbegli neželjeni efekti (promena boja drveća u drugoj polovini video snimka)
2. Zbog toga što konverzija slike iz jednog u drugi kolor sistem oduzima najviše vremena, kao i zbog malih dimenzija snimka, preskočen je prenos detalja sa originalnog frejma na dezen.

## Izbor frejmova za obradu
Za izbor frejmova snimka koje treba obraditi korišćena su dva statistička parametra frejma:
1. Broj kolona maske koje imaju vrednost True
2. Odnos broja elemenata maske koji imaju vrednost True u sredini slike u odnosu na levu i desnu stranu slike

Broj kolona maske sa nenultim vrednostima je adekvatan parametar za određivanje frejmova za obradu zato što je širina kraljice u svim frejmovima približno konstantna. Sa druge strane, frejmovi koji ne sadrže kraljicu, ali sadrže drveće, bivaju obuhvaćeni maskom zbog sličnih karakteristika piksela, ali je broj kolona maske sa nenultim vrednostima mnogo veći u odnosu na frejmove na kojima se nalazi kraljica. Zbog toga je i ovaj parametar iskorišćen kao indikator za primenu maske.

U svakom frejmu u kome se nalazi, kraljica je u centru slike. Zbog toga se svaki frejm može podeliti na 3 dela, i posmatrati odnos broja nenultih elemenata maske u centralnom delu, u odnosu na delove sa strane. Ovim putem takođe se mogu odrediti frejmovi u kojima se nalazi kraljica, i primeniti maska samo na njih.

U sledećem delu koda realizovana je funkcija *dress_queen_video* koja vrši obradu zadatog video snimka, i čuva rezultat pod imenom *new_queens_coat.mp4*.

In [ ]:
# Import modules
from pylab import *
import imageio
from scipy import ndimage

def dress_queen_video():
    """
    Process video to change pattern of Queen's coat.
    """
    # Read video
    filename='../sekvence/queen_coat.mp4'
    vid=imageio.get_reader(filename, 'ffmpeg')

    # Read pattern
    pattern = io.imread('blue_fabric_video.jpg')
    patternYUV=color.rgb2yuv(pattern) # Convert to YUV

    # Define output video object
    video_out = imageio.get_writer('new_queens_coat.mp4', format='FFMPEG', mode = 'I', fps = 30, codec = 'h264')

    # Define filter
    filt = np.ones((15,15))/(15**2)

    for i, cur_frame in enumerate(vid):
        frameHSV = color.rgb2hsv(cur_frame) # Convert to HSV
        frameHSV[:,:,1] = ndimage.correlate(frameHSV[:,:,1], filt) # Apply filter

        # Create image mask
        maskH = (frameHSV[:,:,0]>0.18) & (frameHSV[:,:,0]<0.31)
        maskS1 = (frameHSV[:,:,1]>0.40)
        mask = maskH & maskS1
        height, width = mask.shape

        # Create processing flags

        left = mask[:, :int(width/3)]
        mid = mask[:, int(width/3):int(width*2/3)]
        right = mask[:, int(width*2/3):]
        flag1 = (np.sum(mid)>np.sum(left)) and (np.sum(mid)>np.sum(right))

        numHoriz= np.count_nonzero(np.sum(mask,axis=0))
        flag2 = numHoriz>=120 and numHoriz<305

        flag = flag1 and flag2

        new_frame=cur_frame.copy() # Create new frame

        # Process image if flag
        if (flag):
            new_frame[maskH & maskS1] = pattern[maskH & maskS1]

        video_out.append_data(new_frame) # Append to output video

    video_out.close()

dress_queen_video()

Za usrednjavanje S ravni slike izabran je *box* filtar dimenzija 15x15. Iako generalno ima lošije performanse od Gausovog filtra, u ovom slučaju *box* filtar daje zadovoljavajuće rezultate, i izabran je zbog svoje jednostavnosti. Pragove za definisanje maske nije bilo lako izabrati, jer je u obradi video snimka izbor mnogo teži zbog posmatranja većeg broja frejmova. Posmatranjem histograma više različitih frejmova izabrani su gore navedeni pragovi, koji postižu ne savršen, ali zadovoljavajuć rezultat. Izbor pragova za *flag2* načinjen je sa pretpostavkom da kraljica zauzima najmanje oko $\frac{1}{5}$ frejma, a najviše oko $\frac{1}{2}$ frejma. Ovakvim izborom, u kombinaciji sa *flag1*, dobijaju se odlični rezultati za ovaj video snimak.

# Zadatak 2
U drugom zadatku potrebno je izoštriti sliku *miner.jpg*, prikazati i objasniti postupak izoštravanja, i prikazati rezultate. Za rešavanje ovog zadatka korišćene su dve metode:
1. Primena Laplasijana
2. *Unsharp Masking*

Obe metode su ispitane zasebno i prikazani su rezultati. Na kraju, izveden je zaključak o ovim metodama.

## Cilj
Cilj je dobiti izoštrenu sliku, ali istovremeno izbeći preveliko pojačanje šuma, kao i preveliko pojačanje detalja (oni postaju neprirodni). Ovo se može ostvariti adekvatnim izborom parametara u gore navedenim metodama, što će biti kasnije prikazano.

## Izbor kolor sistema
Za izoštravanje slike *miner.jpg* korišćen je YUV kolor sistem, odnosno njegova Y komponenta. Ova komponenta sadrži sivu sliku sa jasno izraženim detaljima (mnogo bolju od, na primer, V komponente HSV kolor sistema). Primenom operacija nad Y komponentom dobiće se izoštreni detalji, koji će biti vidljivi i nakon konverzije slike u RGB kolor sistem i prikaza slike u boji.

U sledećem delu koda prikazana je slika *miner.jpg*, kao i njena Y komponenta u YUV kolor sistemu.

In [ ]:
# Import image
data_dir = '../sekvence'
imgRGB = io.imread(data_dir + '/miner.jpg')

# Show image and it's Y plane
imgYUV = color.rgb2yuv(imgRGB)
fig, axs=plt.subplots(1,2,figsize=(15, 8), dpi=80)
plt.tight_layout()
axs[0].imshow(imgRGB)
axs[0].set_title('RGB image')
axs[0].set_axis_off()
axs[1].imshow(imgYUV[:,:,0], cmap='gray', vmin=0, vmax=1);
axs[1].set_title('Y plane')
axs[1].set_axis_off();

# Primena Laplasijana
Prva razmatrana metoda za izoštravanje jeste primena Laplasijana. Matematički, Laplasijan se može definisati kao:
$$\nabla ^2f(x,y)=\frac{\partial ^2f}{\partial x^2} + \frac{\partial ^2f}{\partial y^2}$$
odnosno zbir parcijalnih izvoda po obe ose. Primenjen na diskretni signal kao što je slika, Laplasijan se može matematički izraziti kao:
$$\nabla ^2f[x,y]=f[x+1,y]+f[x-1,y]+f[x,y+1]+f[x,y-1]-4f[x,y]$$

Rezultat primene ove formule za izračunavanje Laplasijana ekvivalentna je rezultatu korelacije slike i sledeće matrice:
$$\begin{bmatrix}
0 & 1 & 0\\
1 & -4 & 1\\
0 & 1 & 0
\end{bmatrix}$$

Za veći efekat izoštravanja mogu se uvesti i dijagonalni elementi, pri čemu bi matrica izgledala ovako:
$$\begin{bmatrix}
1 & 1 & 1\\
1 & -8 & 1\\
1 & 1 & 1
\end{bmatrix}$$

Treba naglasiti da zbir svih elemenata matrice treba da bude jednak 0. Ovo pravilo je u skladu sa osobinom izvoda, da na uniformnoj površini (sve vrednosti piksela istog intenziteta) korelacija slike i matrice daje vrednost 0 (nema promene intenziteta).

Laplasijan slike predstavlja detalje, koji se, skalirani nekim faktorom, dodaju na originalnu sliku:
$$g[x,y]=f[x,y]+c[\nabla ^2f[x,y] ]$$
gde $c$ predstavlja faktor skaliranja detalja. Premala vrednost konstante $c$ može dovesti do toga da izoštravanje bude neprimetno, a prevelika vrednost može dovesti do prevelikog pojačanja šuma, ili neprirodno izoštrenih ivica.

U sledećem delu koda primenjene su gore navedene maske, i dobijeni Laplasijani sabrani sa originalnom slikom, nakon množenja različitim faktorima. Takođe, prikazani su dobijeni rezultati.

In [ ]:
# Define matrices
laplaceMask1 = np.array([[0, -1, 0],[-1, 4, -1],[0, -1, 0]])
laplaceMask2 = np.array([[-1, -1, -1],[-1, 8, -1],[-1, -1, -1]])

#Extract details
laplaceDetails1 = ndimage.correlate(imgYUV[:,:,0],laplaceMask1)
laplaceDetails2 = ndimage.correlate(imgYUV[:,:,0],laplaceMask2)

# Plot details
fig, axs = plt.subplots(1,2,figsize=(15, 8), dpi=100)
plt.tight_layout()
axs[0].imshow(laplaceDetails1, cmap='gray', vmin=0, vmax=1)
axs[0].set_axis_off()
axs[0].set_title('Laplacian without diagonal elements')
axs[1].imshow(laplaceDetails2, cmap='gray', vmin=0, vmax=1)
axs[1].set_axis_off()
axs[1].set_title('Laplacian with diagonal elements')

# Add details to original image
imgLaplace1 = imgYUV[:,:,0] + laplaceDetails1
imgLaplace2 = imgYUV[:,:,0] + laplaceDetails2

# Plot grayscale image with enhanced edges
fig, axs = plt.subplots(1,2,figsize=(15, 8), dpi=100)
plt.tight_layout()
axs[0].imshow(imgLaplace1, cmap='gray', vmin=0, vmax=1)
axs[0].set_axis_off()
axs[0].set_title('Grayscale image - Laplacian without diagonal elements')
axs[1].imshow(imgLaplace2, cmap='gray', vmin=0, vmax=1)
axs[1].set_axis_off()
axs[1].set_title('Grayscale image - Laplacian with diagonal elements')

# Convert to RGB
imgYUVLaplace1 = imgYUV.copy()
imgYUVLaplace2 = imgYUV.copy()
imgYUVLaplace1[:,:,0]=imgLaplace1
imgYUVLaplace2[:,:,0]=imgLaplace2
imgRGBLaplace1=color.yuv2rgb(imgYUVLaplace1)
imgRGBLaplace2=color.yuv2rgb(imgYUVLaplace2)

imgRGBLaplace1 = np.clip(imgRGBLaplace1, 0, 1)
imgRGBLaplace2 = np.clip(imgRGBLaplace2, 0, 1)

# Plot final results
fig, axs = plt.subplots(1,2,figsize=(15, 8), dpi=100)
plt.tight_layout()
axs[0].imshow(imgRGBLaplace1, cmap='gray', vmin=0, vmax=1)
axs[0].set_axis_off()
axs[0].set_title('RGB image - Laplacian without diagonal elements')
axs[1].imshow(imgRGBLaplace2, cmap='gray', vmin=0, vmax=1)
axs[1].set_axis_off()
axs[1].set_title('RGB image - Laplacian with diagonal elements');

Očigledno je da maska sa dijagonalnim elementima previše naglašava ivice, i previše pojačava šum. To se najviše vidi na licu rudara, koje postaje previše zašumljeno. Sa druge strane, slika dobijena primenom maske bez dijagonalnih elemenata izgleda zadovoljavajuće - detalji su izraženi, ali nisu prenaglašeni, i šum nije previše pojačan. U daljem razmatranju biće korišćena ta maska, ali će detalji biti skalirani različitim vrednostima faktora $c$. Cilj je dobiti malo naglašenije ivice, bez previše vidljivog šuma.

U sledećem delu koda realizovana je primena Laplasijana za različite vrednosti skaliranja, i prikazani su rezultati.

In [ ]:
# Add scaled details to original image
imgLaplace1 = imgYUV[:,:,0] + laplaceDetails1
imgLaplace2 = imgYUV[:,:,0] + 1.2*laplaceDetails1
imgLaplace3 = imgYUV[:,:,0] + 1.5*laplaceDetails1

# Clip to range (0,1)
imgLaplace2 = np.clip(imgLaplace2, 0, 1)
imgLaplace3 = np.clip(imgLaplace3, 0, 1)

# Plot grayscale image with enhanced edges
fig, axs = plt.subplots(1,3,figsize=(15, 8), dpi=90)
plt.tight_layout()
axs[0].imshow(imgLaplace1, cmap='gray', vmin=0, vmax=1)
axs[0].set_axis_off()
axs[0].set_title('Grayscale image: c=1')
axs[1].imshow(imgLaplace2, cmap='gray', vmin=0, vmax=1)
axs[1].set_axis_off()
axs[1].set_title('Grayscale image: c=1.2')
axs[2].imshow(imgLaplace3, cmap='gray', vmin=0, vmax=1)
axs[2].set_axis_off()
axs[2].set_title('Grayscale image: c=1.5');

# Convert to RGB
imgYUVLaplace1 = imgYUV.copy()
imgYUVLaplace2 = imgYUV.copy()
imgYUVLaplace3 = imgYUV.copy()
imgYUVLaplace1[:,:,0]=imgLaplace1
imgYUVLaplace2[:,:,0]=imgLaplace2
imgYUVLaplace3[:,:,0]=imgLaplace3
imgRGBLaplace1=color.yuv2rgb(imgYUVLaplace1)
imgRGBLaplace2=color.yuv2rgb(imgYUVLaplace2)
imgRGBLaplace3=color.yuv2rgb(imgYUVLaplace3)

imgRGBLaplace1 = np.clip(imgRGBLaplace1, 0, 1)
imgRGBLaplace2 = np.clip(imgRGBLaplace2, 0, 1)
imgRGBLaplace3 = np.clip(imgRGBLaplace3, 0, 1)

# Plot final results
fig, axs = plt.subplots(1,3,figsize=(15, 8), dpi=90)
plt.tight_layout()
axs[0].imshow(imgRGBLaplace1, cmap='gray', vmin=0, vmax=1)
axs[0].set_axis_off()
axs[0].set_title('RGB image: c=0')
axs[1].imshow(imgRGBLaplace2, cmap='gray', vmin=0, vmax=1)
axs[1].set_axis_off()
axs[1].set_title('RGB image: c=1.2')
axs[2].imshow(imgRGBLaplace3, cmap='gray', vmin=0, vmax=1)
axs[2].set_axis_off()
axs[2].set_title('RGB image: c=1.5');

Na osnovu gore prikazanih slika zaključak je da najbolje rezultate daje faktor skaliranja $c=1.2$: skaliranjem detalja ovim faktorom naglašavaju se detalji u toj meri da oni ne budu prenaglašeni, i da se šum ne pojača previše. Faktor skaliranja $c=1.5$ daje prenaglašene detalje, što se vidi na slici. Zaključak je da se metodom Laplasijana najbolja izoštrena slika *imgRGBLaplace2*, dobijena primenom Laplasijana bez dijagonalnih elemenata, i pomnoženim faktortom skaliranja $c=1.2$.

## *Unsharp Masking*
*Unsharp Masking*, odnosno metoda isticanja visokih učestanosti, jeste metoda koja detalje izdvaja oduzimajući zamućenu sliku od originalne. Dalje, ti detalji se dodaju na originalnu sliku, čime se slika izoštrava. Matematički zapis ovog postupka je:
$$g_{mask}[x, y]=f[x,y]-\overline{f}[x,y]$$
$$g[x, y]=f[x,y]+kg_{mask}[x,y]$$
gde je:
* $\overline{f}[x,y]$ usrednjena slika
* $g_{mask}[x, y]$ dobijena maska sa detaljima
* $k$ faktor skaliranja detalja
* $g[x,y]$ konačan rezultat

Metod *Unsharp Masking* može se primeniti u jednom koraku korelacijom originalne slike sa nekom od sledećih matrica:
$$\begin{align}
um_1 &=\begin{bmatrix}
0 & -1 & 0\\
-1 & 5 & -1\\
0 & -1 & 0
\end{bmatrix}\\
um_2 &=\begin{bmatrix}
1 & -2 & 1\\
-2 & 5 & -2\\
1 & -2 & 1
\end{bmatrix}\\
um_3 &=\begin{bmatrix}
-1 & -2 & -1\\
-2 & 19 & -2\\
-1 & -2 & -1
\end{bmatrix}\frac{1}{7}
\end{align}$$


pri čemu ove matrice predstavljaju razliku matrice identiteta i različitih matrica za detekciju ivica.

U sledećem delu koda primenjene su gore navedene matrice na originalnu sliku i izneti su rezultati ove metode.

In [ ]:
# Define filters
um1=np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
um2=np.array([[1, -2, 1], [-2, 5, -2], [1, -2, 1]])
um3=np.array([[-1, -2, -1], [-2, 19, -2], [-1, -2, -1]])/7

# Apply filters
umDetails1 = ndimage.correlate(imgYUV[:,:,0], um1)
umDetails2 = ndimage.correlate(imgYUV[:,:,0], um2)
umDetails3 = ndimage.correlate(imgYUV[:,:,0], um3)

# Plot details
fig, axs = plt.subplots(1,3,figsize=(15, 8), dpi=90)
plt.tight_layout()
axs[0].imshow(umDetails1, cmap='gray', vmin=0, vmax=1)
axs[0].set_axis_off()
axs[0].set_title('Unsharp masking: mask 1')
axs[1].imshow(umDetails2, cmap='gray', vmin=0, vmax=1)
axs[1].set_axis_off()
axs[1].set_title('Unsharp masking: mask 2')
axs[2].imshow(umDetails3, cmap='gray', vmin=0, vmax=1)
axs[2].set_axis_off()
axs[2].set_title('Unsharp masking: mask 3');

# Convert to RGB
imgYUVum1 = imgYUV.copy()
imgYUVum2 = imgYUV.copy()
imgYUVum3 = imgYUV.copy()
imgYUVum1[:,:,0]=imgLaplace1
imgYUVum2[:,:,0]=imgLaplace2
imgYUVum3[:,:,0]=imgLaplace3
imgRGBum1=color.yuv2rgb(imgYUVLaplace1)
imgRGBum2=color.yuv2rgb(imgYUVLaplace2)
imgRGBum3=color.yuv2rgb(imgYUVLaplace3)

imgRGBum1 = np.clip(imgRGBum1, 0, 1)
imgRGBum2 = np.clip(imgRGBum2, 0, 1)
imgRGBum3 = np.clip(imgRGBum3, 0, 1)

# Plot final results
fig, axs = plt.subplots(1,3,figsize=(15, 8), dpi=90)
plt.tight_layout()
axs[0].imshow(imgRGBum1, cmap='gray', vmin=0, vmax=1)
axs[0].set_axis_off()
axs[0].set_title('RGB image: mask 1')
axs[1].imshow(imgRGBum2, cmap='gray', vmin=0, vmax=1)
axs[1].set_axis_off()
axs[1].set_title('RGB image: mask 2')
axs[2].imshow(imgRGBum3, cmap='gray', vmin=0, vmax=1)
axs[2].set_axis_off()
axs[2].set_title('RGB image: mask 3');

Rezultati pokazuju da maska *um1* daje najbolje rezultate: pojačava ivice (ali ih ne prenaglašava) i istovremeno ne pojačava previše šum (kao što je slučaj kod ostalih maski). Ova maska dobijena je kao razlika matrice identiteta
$$\begin{bmatrix}
0 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 0
\end{bmatrix}$$
i matrice za detekciju ivica
$$\begin{bmatrix}
0 & 1 & 0\\
1 & -4 & 1\\
0 & 1 & 0
\end{bmatrix}$$
gde treba primetiti da je ova matrica jednaka ranije ispitivanoj matrici Laplasijana, sa promenjenim znakovima elemenata.

Na kraju, treba proveriti kakve rezultate daje primena ovog metoda, sa skaliranim vrednostima matrice za detekciju ivica. Kao i ranije, cilj je izbeći prenaglašavanje ivica i preveliko pojačanje šuma.

U sledećem delu koda ispitan je uticaj faktora skaliranja na konacan rezultat, i izloženi su finalni rezultati.

In [ ]:
identityMatrix = np.zeros((3,3))
identityMatrix[1,1]=1
edgeDetectionMatrix=np.array([[0, 1, 0],[1, -4, 1],[0, 1, 0]])
um1=identityMatrix-edgeDetectionMatrix
um2=identityMatrix-1.2*edgeDetectionMatrix
um3=identityMatrix-1.5*edgeDetectionMatrix

# Apply Unsharp Masking
imgUM1=ndimage.correlate(imgYUV[:,:,0], um1)
imgUM2=ndimage.correlate(imgYUV[:,:,0], um2)
imgUM3=ndimage.correlate(imgYUV[:,:,0], um3)

 # Clip to range (0, 1)
imgUM1 = np.clip(imgUM1, 0, 1)
imgUM2 = np.clip(imgUM2, 0, 1)
imgUM3 = np.clip(imgUM3, 0, 1)

# Plot grayscale image with enhanced edges
fig, axs = plt.subplots(1,3,figsize=(15, 8), dpi=90)
plt.tight_layout()
axs[0].imshow(imgUM1, cmap='gray', vmin=0, vmax=1)
axs[0].set_axis_off()
axs[0].set_title('Grayscale image: c=1')
axs[1].imshow(imgUM2, cmap='gray', vmin=0, vmax=1)
axs[1].set_axis_off()
axs[1].set_title('Grayscale image: c=1.2')
axs[2].imshow(imgUM3, cmap='gray', vmin=0, vmax=1)
axs[2].set_axis_off()
axs[2].set_title('Grayscale image: c=1.5');

# Convert to RGB
imgYUVum1 = imgYUV.copy()
imgYUVum2 = imgYUV.copy()
imgYUVum3 = imgYUV.copy()
imgYUVum1[:,:,0]=imgUM1
imgYUVum2[:,:,0]=imgUM2
imgYUVum3[:,:,0]=imgUM3
imgRGBum1=color.yuv2rgb(imgYUVum1)
imgRGBum2=color.yuv2rgb(imgYUVum2)
imgRGBum3=color.yuv2rgb(imgYUVum3)

imgRGBum1 = np.clip(imgRGBum1, 0, 1)
imgRGBum2 = np.clip(imgRGBum2, 0, 1)
imgRGBum3 = np.clip(imgRGBum3, 0, 1)

# Plot final results
fig, axs = plt.subplots(1,3,figsize=(15, 8), dpi=90)
plt.tight_layout()
axs[0].imshow(imgRGBum1, cmap='gray', vmin=0, vmax=1)
axs[0].set_axis_off()
axs[0].set_title('RGB image: c=0')
axs[1].imshow(imgRGBum2, cmap='gray', vmin=0, vmax=1)
axs[1].set_axis_off()
axs[1].set_title('RGB image: c=1.2')
axs[2].imshow(imgRGBum3, cmap='gray', vmin=0, vmax=1)
axs[2].set_axis_off()
axs[2].set_title('RGB image: c=1.5')

# Plot final comparison
fig, axs = plt.subplots(1,3,figsize=(15, 8), dpi=90)
plt.tight_layout()
axs[0].imshow(imgRGB, cmap='gray', vmin=0, vmax=1)
axs[0].set_axis_off()
axs[0].set_title('Original image')
axs[1].imshow(imgRGBLaplace2, cmap='gray', vmin=0, vmax=1)
axs[1].set_axis_off()
axs[1].set_title('Enchancement method: Laplacian')
axs[2].imshow(imgRGBum2, cmap='gray', vmin=0, vmax=1)
axs[2].set_axis_off()
axs[2].set_title('Enchancement method: Unsharp Masking')

io.imsave('miner_sharp.jpg',util.img_as_ubyte(imgRGBum2))

print(np.max(np.abs(imgLaplace2-imgUM2)))

Rezultati pokazuju da je vrednost konstante $c=1.2$ optimalna za dovoljno izoštravanje slike tako da ne dođe do prenaglašenja ivica i prevelikog pojačanja šuma.

## Zaključak
Primenom obe navedene metode dobijeni su zadovoljavajući rezultati. Poređenjem dobijenih slika istaknuto je da je maksimalna apsolutna razlika intenziteta piksela crno-belih slika dobijenih primenom obe metode izuzetno mala, što daje jedan veoma bitan zaključak: ove dve metode mogu za adekvatan izbor parametara davati isti rezultat, odnosno ekvivalentne su. 

# Zadatak 3
U trećem zadatku potrebno je definisati funkciju *imgOut = dosCLAHE(imgIn, numTiles, limit)* koja realizuje adaptivnu ekvalizaciju histograma uz ograničenje kontrasta, bez upotrebe ugrađenih funkcija za određivanje histograma, ekvalizaciju histograma i interpolaciju slike. Pored tačnih rezultata, glavna odlika ove funkcije treba biti kratko vreme izvršavanja. Vreme izvršavanja se može višestruko smanjiti primenom matričnih operacija umesto petlji. Na taj način se mnogo manje vremena troši na interpretaciju koda, što daje rezultat za izuzetno kratko vreme. Iako ovaj način zahteva više memorije, vreme izvršavanja je u ovom slučaju bitnije, pa će rešenje tako biti i realizovano.

## Pomoćne funkcije

Rešavanje ovog zadatka može se podeliti u više celina. Pre svega, neophodno je definisati gore navedene pomoćne funkcije koje će biti korišćene unutar funkcije *dosCLAHE*. Pored njih, definisane su i dodatne pomoćne funkcije radi podele koda na logičke celine.

U sledećem delu koda uvezeni su potrebni moduli koji do sad nisu korišćeni i definisane su pomoćne funkcije korišćene unutar funkcije *dosCLAHE*.

In [ ]:
# Import modules
import time
from skimage import exposure

# Define helper functions
def dosHistogram(arr):
    """
        Calculate histogram of input array.
        
        Parameters
        ----------
        arr: ndarray
            Input array.
            
        Returns
        -------
        hist: ndarray
            Array of size 256, representing calculated histogram.
        bins: ndarray
            Array of size 256, representing histogram bins edges.
        
    """
    hist = np.zeros(256, dtype=int)
    for pixel in arr:
        hist[pixel] += 1
    return hist, np.arange(256)

def dosGetCdf(img, limit=0.01):
    """
        Calculate Cumulative Density Function of an image, with limited Probability Density Function.
        
        Parameters
        ----------
        img: ndarray
            Input image represented by 2D array.
        limit: integer
            Number of horizontal and vertical blocks.
        limit: int
            Maximum PDF value allowed.
        
        Returns
        -------
        cdf: ndarray
            Cumulative Distribution Function values.
        
    """
    imgHistogram, edges = dosHistogram(img.flatten()) # get image histogram
    pdf_nolimit = imgHistogram/img.size # calculate pdf without contrast limit

    pdf=np.clip(pdf_nolimit, 0, limit) # find PDF with contrast limit
    

    # Add the difference to pdf to make it's area equal to 1
    difference = 1-np.sum(pdf)
    pdf = pdf + np.array([difference/pdf.size]*pdf.size)

    # Calculate CDF from PDF
    cdf = np.clip(np.cumsum(pdf),0, 1)

    return cdf

def dosAddPadding(img, numTiles):
    """
        Add padding to input image, to make new image divisable by corresponding numTiles values.
        
        Parameters
        ----------
        imgIn: ndarray
            Input image.
        numTiles: array_like or tuple
            Number of horizontal and vertical blocks.
        limit: int
            Clip Limit.
        
        Returns
        -------
        paddedImg: ndarray
            Image with the same size as imgIn, processed using CLAHE method.
        upperPad: int
            Index of the original image starting row.
        leftPad: int
            Index of the original image starting column.
    """
    imgHeight, imgWidth = img.shape # get image shape

    # Calculate upper and lower padding size
    horizPad = (numTiles[0] - (imgHeight % numTiles[0])) % numTiles[0]
    upperPad = int(horizPad/2) # if horizPad is odd, lowerPad=upperPad+1
    lowerPad = horizPad - upperPad

    # Calculate left and right padding size
    vertPad = (numTiles[1] - (imgWidth % numTiles[1])) % numTiles[1]
    leftPad = int(vertPad/2)  #if vertPad is odd, rightPad=leftPad+1
    rightPad = vertPad - leftPad

    # Create padded image
    paddedImg = np.zeros((imgHeight+horizPad, imgWidth+vertPad), dtype=np.uint8)
    paddedImg[:upperPad, leftPad:imgWidth+leftPad] = img[:upperPad, :]
    paddedImg[imgHeight+upperPad:, leftPad:imgWidth+leftPad] = img[imgHeight-lowerPad:, :]
    paddedImg[upperPad:imgHeight+upperPad, :leftPad] = img[:, :leftPad]
    paddedImg[upperPad:imgHeight+upperPad, imgWidth+leftPad:] = img[:, imgWidth-rightPad:]
    paddedImg[upperPad:imgHeight+upperPad, leftPad:imgWidth+leftPad] = img

    # Return padded image and original image start indices
    return paddedImg, upperPad, leftPad

def dosCheckInputParams(imgIn, numTiles, limit):
    """
        Check function input parameters.
        
        Parameters
        ----------
        img: ndarray
            Input image.
        numTiles: array-like
            Number of horizontal and vertical blocks.
            
        Returns
        -------
        ok: bool
            True if all input parameters are as expected, otherwise False.
    """
    cond1 = imgIn is not None # check if image is properly loaded
    cond2 = (imgIn.ndim == 2) or (imgIn.ndim == 3) # check image dims
    if(imgIn.ndim == 2):
        cond3 = isinstance(imgIn[0,0], np.uint8) # check image values type (grayscale)
    elif(imgIn.ndim == 3):
        cond3 = isinstance(imgIn[0,0,0], np.uint8) # check image values type (RGB)
    cond4 = len(numTiles)==2 # check numTiles dims
    cond5 = (limit>=0)

    return (cond1 and cond2 and cond3 and cond4 and cond5)

def dosCLAHEgrayscale(img, numTiles=[8,8], limit=0.01):
    """
        Perform Adaptive Histogram Equalization with Contrast Limit on Grayscale image.
        This is a helper function used in dosCLAHE(imgIn, numTiles, limit).
        
        Parameters
        ----------
        imgIn: ndarray
            Input Grayscale image. Pixel intensity should be in range (0,255).
        numTiles: array_like
            Number of horizontal and vertical blocks.
        limit: int
            Clip Limit.
        
        Returns
        -------
        imgOut: ndarray
            Image with the same size as imgIn, processed using CLAHE method.
    """
    
    # Add padding
    paddedImg, upperPad, leftPad= dosAddPadding(img, numTiles)
    
     # Get dims
    imgHeight = img.shape[0]
    imgWidth = img.shape[1]
    paddedHeight = paddedImg.shape[0]
    paddedWidth = paddedImg.shape[1]
    tileHeight = int(paddedHeight/numTiles[0])
    tileWidth = int(paddedWidth/numTiles[1])
    
    # Get pixel coordinate matrices
    xGrid, yGrid = np.indices((paddedHeight, paddedWidth))

    # Get all tiles coordinate matrices
    tileXgrid = xGrid//tileHeight
    tileYgrid = yGrid//tileWidth

    # Get central pixel coordinate matrices
    xc = np.ceil(tileHeight*np.floor(xGrid/tileHeight) + (tileHeight-1)/2).astype(int)
    yc = np.ceil(tileWidth*np.floor(yGrid/tileWidth) + (tileWidth-1)/2).astype(int)
    
    # Get corresponding blocks coordinate matrices

    # Define matrices
    Px = np.zeros((paddedHeight, paddedWidth), dtype=int)
    Py = np.zeros((paddedHeight, paddedWidth), dtype=int)
    Qx = np.zeros((paddedHeight, paddedWidth), dtype=int)
    Qy = np.zeros((paddedHeight, paddedWidth), dtype=int)
    Rx = np.zeros((paddedHeight, paddedWidth), dtype=int)
    Ry = np.zeros((paddedHeight, paddedWidth), dtype=int)
    Sx = np.zeros((paddedHeight, paddedWidth), dtype=int)
    Sy = np.zeros((paddedHeight, paddedWidth), dtype=int)

    # Fill blocks matrices
    case1 = (xGrid-xc<0) & (yGrid-yc<0)
    Px[case1] = (tileXgrid-1)[case1]
    Py[case1] = (tileYgrid-1)[case1]
    Qx[case1] = (tileXgrid-1)[case1]
    Qy[case1] = tileYgrid[case1]
    Rx[case1] = tileXgrid[case1]
    Ry[case1] = tileYgrid[case1]
    Sx[case1] = tileXgrid[case1]
    Sy[case1] = (tileYgrid-1)[case1]

    case2 = (xGrid-xc>=0) & (yGrid-yc<0)
    Px[case2] = tileXgrid[case2]
    Py[case2] = (tileYgrid-1)[case2]
    Qx[case2] = tileXgrid[case2]
    Qy[case2] = tileYgrid[case2]
    Rx[case2] = (tileXgrid+1)[case2]
    Ry[case2] = tileYgrid[case2]
    Sx[case2] = (tileXgrid+1)[case2]
    Sy[case2] = (tileYgrid-1)[case2]

    case3 = (xGrid-xc<0) & (yGrid-yc>=0)
    Px[case3] = (tileXgrid-1)[case3]
    Py[case3] = tileYgrid[case3]
    Qx[case3] = (tileXgrid-1)[case3]
    Qy[case3] = (tileYgrid+1)[case3]
    Rx[case3] = tileXgrid[case3]
    Ry[case3] = (tileYgrid+1)[case3]
    Sx[case3] = tileXgrid[case3]
    Sy[case3] = tileYgrid[case3]

    case4 = (xGrid-xc>=0) & (yGrid-yc>=0)
    Px[case4] = tileXgrid[case4]
    Py[case4] = tileYgrid[case4]
    Qx[case4] = tileXgrid[case4]
    Qy[case4] = (tileYgrid+1)[case4]
    Rx[case4] = (tileXgrid+1)[case4]
    Ry[case4] = (tileYgrid+1)[case4]
    Sx[case4] = (tileXgrid+1)[case4]
    Sy[case4] = (tileYgrid+1)[case4]

    # Clip values to proper range
    PxClipped = np.clip(Px, 0, numTiles[0]-1)
    PyClipped = np.clip(Py, 0, numTiles[1]-1)
    QxClipped = np.clip(Qx, 0, numTiles[0]-1)
    QyClipped = np.clip(Qy, 0, numTiles[1]-1)
    RxClipped = np.clip(Rx, 0, numTiles[0]-1)
    RyClipped = np.clip(Ry, 0, numTiles[1]-1)
    SxClipped = np.clip(Sx, 0, numTiles[0]-1)
    SyClipped = np.clip(Sy, 0, numTiles[1]-1)
    
    # Get CDF of every Tile
    CDFs = np.zeros((numTiles[0], numTiles[1], 256), dtype=float)
    for i in range(numTiles[0]):
        for j in range(numTiles[1]):
            CDFs[i,j,:] = dosGetCdf(paddedImg[i*tileHeight:(i+1)*tileHeight, j*tileWidth:(j+1)*tileWidth], limit)
            
    # Get corresponding CDF values matrices
    Tp=255*CDFs[PxClipped,PyClipped, paddedImg[xGrid, yGrid]]
    Tq=255*CDFs[QxClipped,QyClipped, paddedImg[xGrid, yGrid]]
    Tr=255*CDFs[RxClipped,RyClipped, paddedImg[xGrid, yGrid]]
    Ts=255*CDFs[SxClipped,SyClipped, paddedImg[xGrid, yGrid]]
    
    # Calculate scale factors for bilinear interpolation
    a = np.abs(yGrid-(Py*tileWidth+tileWidth//2))
    b = tileWidth-a
    c = np.abs(xGrid-(Px*tileHeight+tileHeight/2))
    d = tileHeight-c
    
    # Calculate final pixel intensity matrix
    sh1 = (b*Tp + a*Tq)/(a+b)
    sh2 = (b*Ts + a*Tr)/(a+b)
    imgOut = ((d*sh1 + c*sh2)/(c+d))
    
    # Return image without padding
    return imgOut[upperPad:imgHeight+upperPad,leftPad:imgWidth+leftPad]
    
    

## Funkcija dosCLAHE(imgIn, numTiles, limit)
Ova funkcija može se podeliti u nekoliko celina, kao što je to urađeno definisanjem pomoćnih funkcija. Za početak, neophodno je proveriti validnost unetih parametara pozivom funkcije *checkInputParams(imgIn, numTiles, limit)* i prekinuti izvršavanje ukoliko neki od parametara nije validan. Dalje, u zavisnosti od toga da li je ulazna slika siva ili u boji, poziva se funkcija *dosCLAHEgrayscale(imgIn, numTiles, limit)* na sivu sliku, ili na Y komponentu YUV slike, u slučaju da je na ulazu prosleđena slika u boji. Sam metod definisan je tako da se petlje maksimalno izbegavaju, čime se dobija višestruko manje vreme izvršavanja funkcije.

U sledećem delu koda definisana je funkcija *dosCLAHE(imgIn, numTiles, limit)*.

In [ ]:
def dosCLAHE(imgIn, numTiles=[8, 8], limit=0.01):
    """
        Perform Adaptive Histogram Equalization with Contrast Limit.
        This function is a wrapper function of dosCLAHEgrayscale, which implements CLAHE method.
        
        Parameters
        ----------
        imgIn: ndarray
            Input image. If it's type is 3D the image is considered to be in RGB format. Values are of type uint8.
        numTiles: array_like
            Number of horizontal and vertical blocks.
        limit: int
            Clip Limit.
        
        Returns
        -------
        imgOut: ndarray
            Image with the same size as imgIn, processed using CLAHE method.
    """
    
    # Check input parameters
    if (not dosCheckInputParams(imgIn, numTiles, limit)):
        print('Wrong Input')
        return None
    
   
    
    if(imgIn.ndim==2): # Grayscale image
        imgOut = dosCLAHEgrayscale(imgIn, numTiles, limit)
    else: # RGB image
        imgYUV = color.rgb2yuv(img)
        eqImgYUV=imgYUV.copy()
        eqImgYUV[:,:,0]=dosCLAHEgrayscale(255*imgYUV[:,:,0],numTiles,limit)/255
        imgOut=color.yuv2rgb(eqImgYUV)
            
    return util.img_as_ubyte(np.clip(imgOut,0,1))
            

## Testiranje funkcije *dosCLAHE(imgIn, numTiles, limit)*
Gore definisanu funkciju potrebno je testirati na slici *train.jpg* za različit broj blokova i različite vrednosti limita. Treba odrediti uticaj parametara na vreme izvršavanja, kao i na krajnji rezultat. Treba normalizovati vreme izvršavanja sa brojem piksela slike, i zavisnost vremena od parametara prikazati grafički. Na kraju, potrebno je uporediti rezultate i vreme izvršavanja sa ugrađenom funkcijom *exposure.equalize_adapthist*.

Sledeći deo koda obuhvata navedena testiranja i ispis rezultata.

In [ ]:
# Read image
data_dir = '../sekvence'
img = util.img_as_ubyte(io.imread(data_dir + '/train.jpg'))
height, width, c = img.shape

# Define params
numBlocks = [1, 4, 8, 16]
limits = [0, 0.01, 0.1, 1]

# Save measured times
timesCustom=[]
timesBuiltIn=[]

# Scatter Axes
blocksAxis=[]
limitAxis=[]

for num in numBlocks:
    fig, axs = plt.subplots(2, 4,figsize=(20,8), dpi=90)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    ctr = 0
    for lim in limits:
        blocksAxis.append(num)
        limitAxis.append(lim)
        start = time.time()
        imgOut = dosCLAHE(img, numTiles=[num, num], limit=lim)
        end=time.time()
        exec_time = end-start
        print('dosCLAHE                    | ',  [num, num], ' | ', format(lim, '1.2f'), ' | ',  format(exec_time,'1.3f'), "s | ", (exec_time*1e6/img.size), "us/px")
        timesCustom.append(exec_time)
        start = time.time()
        imgOutBuiltIn = exposure.equalize_adapthist(img, kernel_size=[height/num, width/num], clip_limit=lim)
        end=time.time()
        exec_time = end-start
        print('exposure.equalize_adapthist | ',  [num, num], ' | ', format(lim, '1.2f'), ' | ',  format(exec_time,'1.3f'), "s | ", (exec_time*1e6/img.size), "us/px")
        timesBuiltIn.append(exec_time)
        axs[0, ctr].imshow(imgOut)
        axs[0, ctr].set_title(str([num, num]) + ' | ' + str(lim))
        axs[0, ctr].set_axis_off()
        axs[1, ctr].imshow(imgOutBuiltIn)
        axs[1, ctr].set_title(str([num, num]) + ' | ' + str(lim))
        axs[1, ctr].set_axis_off()
        fig.suptitle('Above: dosCLAHE | Below: exposure.equalize_adapthist', fontsize=20)
        ctr+=1

## Uticaj vrednosti limita
Sa prikazanih slika je očigledno da za vrednosti limita 0.1 i 1 slika dodatno gubi na kvalitetu. Razlog tome je što se, povećavanjem vrednosti limita, povećava i maskimalni nagib funkcije raspodele koju koristimo za preslikavanje intenziteta originalne slike. Veliki nagib dovodi do preslikavanja malog opsega intenziteta na veliki opseg, čime se javlja efekat posterizacije - razlike intenziteta okolnih piksela postaju jasno vidljive, čak i kod površina kod kojih je na originalnoj slici ova razlika veoma mala. Sa druge strane, za vrednost limita 0 funkcija raspodele postaje linearna sa nagibom 1, čime praktično nema nikakve promene u odnosu na originalnu sliku. Vrednost limita 0.01 daje najbolje rezultate, bez obzira na vrednost *numTiles*.

## Uticaj broja blokova
Originalna slika ima izražen kontrast, što dovodi do zasićenja svetlih piksela nakon ekvalizacije u slučaju korišćenja malog broja blokova. Ovaj efekat je uočljiv na slici dobijenoj primenom funkcije za vrednost *numTiles* [1, 1] Takođe, loši rezultati se dobijaju i za veliki broj blokova na koje delimo sliku. U tom slučaju blokovi sadrže nedovoljan broj piksela za adekvatno formiranje funkcije raspodele, što dovodi do lošeg mapiranja intenziteta. Kao rezultat javljaju se izraženi regioni koje je nemoguće otkloniti bilinearnom interpolacijom. Dobar rezultat dobija se za *numTiles*=[4, 4], jer se u tom slučaju izdvajaju regioni sa sličnim vrednostima intenziteta piksela, i svaki blok sadrži dovoljan broj piksela za formiranje adekvatne funkcije raspodele.

Najbolji rezultat dobija se za vrednosti *numTiles*=[4,4] i *limit*=0.01.

## Poređenje rezultata sa ugrađenom funkcijom
Zbog načina implementacije ugrađene funkcije *exposure.equalize_adapthist*, koja za ulaznu sliku u boji prvo vrši konverziju u HSV kolor sistem, i radi ekvalizaciju histograma V kanala, rezultati koje daje se veoma razlikuju od rezultata funkcije *dosCLAHE*. Sama V komponenta HSV kolor sistema ne prikazuje detalje dovoljno dobro kao Y komponenta YUV kolor sistema, pa se kao rezultat nakon ekvalizacije dobija prenaglašena plava boja. Uticaj parametara je gotovo isti kao kod funkcije *dosCLAHE*, pa je se najbolji rezultat takođe dobija za parametre *numTiles*=[4,4] i *limit*=0.01.

# Vreme izvršavanja funkcije *dosCLAHE*
U sledećem delu koda prikazana je zavisnost vremena potrebnog za izvršavanje funkcije *dosCLAHE* od njenih argumenata *numTiles* i *limit*.

In [ ]:
fig, ax = plt.subplots(2,figsize=(15, 15), dpi=120)
scatter = ax[0].scatter(blocksAxis, limitAxis, c=timesCustom, s=500, cmap='inferno')

# produce a legend with the unique colors from the scatter
legend1 = ax[0].legend(*scatter.legend_elements(), loc=(1.04,0.2), shadow=True, fontsize=10, title='Time[s]',title_fontsize=10)
ax[0].add_artist(legend1)
ax[0].grid(True)
ax[0].set_yscale('symlog', linthresh=0.015)
ax[0].set_xscale('log', base=2)
ax[0].set_xlabel('$numTiles$', fontsize=20)
ax[0].set_ylabel('$limit$', fontsize=20)
ax[0].set_title('Time to execute $dosCLAHE$', fontsize=20);

scatter = ax[1].scatter(blocksAxis, limitAxis, c=timesBuiltIn, s=500, cmap='inferno')

# produce a legend with the unique colors from the scatter
legend1 = ax[1].legend(*scatter.legend_elements(), loc=(1.04,0.2), shadow=True, fontsize=10, title='Time[s]',title_fontsize=10)
ax[1].add_artist(legend1)
ax[1].grid(True)
ax[1].set_yscale('symlog', linthresh=0.015)
ax[1].set_xscale('log', base=2)
ax[1].set_xlabel('$numTiles$', fontsize=20)
ax[1].set_ylabel('$limit$', fontsize=20)
ax[1].set_title('Time to execute $exposure.equalize\_adapthist$', fontsize=20);

Sa grafika se vidi da se vreme izvršavanja funkcije *dosCLAHE* veoma malo menja u zavisnosti od parametara *numTiles*, dok parametar *limit* nema gotovo nikakav uticaj. Razlog tome jeste činjenica da je funkcija *dosCLAHE* definisana tako da sve velika većina operacija izvršava matrično, dok se petlje izbegavaju. Na taj način se smanjuje vreme potrebno za interpretaciju *Python* koda, čime se višestruko ubrzava izvršavanje. Petlje su korišćene samo za određivanje funkcija raspodele svakog bloka, gde se koriste za iteraciju po blokovima. Upravo to je razlog malo sporijeg izvršavanja u slučaju sa većim brojem blokova.

Zbog drugačije implementacije, vreme izvršavanja ugrađene funkcije smanjuje se sa povećanjem broja blokova. Iako je brža, maksimalna razlika vremena izvršavanja ovih funkcija je manja od 200ms, što je zadovoljavajuć rezultat.